# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Mesothelioma'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [6]:
trait_subdir = "TCGA_Mesothelioma_(MESO)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.MESO.sampleMap_MESO_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.MESO.sampleMap_HiSeqV2_PANCAN.gz')

In [7]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [8]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [9]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'age_at_initial_pathologic_diagnosis',
 'asbestos_exposure_age',
 'asbestos_exposure_age_last']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [40]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'asbestos_exposure_age', 'asbestos_exposure_age_last', 'asbestos_exposure_source', 'asbestos_exposure_type', 'asbestos_exposure_years', 'assessment_timepoint_category', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'creatinine_norm_range_lower', 'creatinine_norm_range_u

In [41]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'asbestos_exposure_age', 'asbestos_exposure_age_last', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'person_neoplasm_cancer_status', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']


Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [42]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [64, 60, 53, 58, 69],
 'asbestos_exposure_age': [nan, nan, nan, 25.0, nan],
 'asbestos_exposure_age_last': [nan, nan, nan, 31.0, nan],
 'days_to_birth': [-23591, -21972, -19503, -21423, -25471],
 'days_to_collection': [1935.0, 1941.0, 1998.0, 1897.0, 1743.0],
 'days_to_death': ['1715', '361', '885', '1302', '385'],
 'days_to_initial_pathologic_diagnosis': [0, 0, 0, 0, 0],
 'days_to_last_followup': [nan, nan, nan, nan, nan],
 'days_to_new_tumor_event_after_initial_treatment': [nan, nan, nan, nan, nan],
 'person_neoplasm_cancer_status': ['WITH TUMOR',
  'WITH TUMOR',
  'WITH TUMOR',
  'WITH TUMOR',
  'WITH TUMOR'],
 'year_of_initial_pathologic_diagnosis': [2008, 2008, 2008, 2008, 2009]}

In [43]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [44]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [45]:
gender_col = 'gender'

In [46]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [47]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

13 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('RPL31P11', 2), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('
12 input query terms found dup hits:	[('SUGT1P1', 2), ('PTPRVP', 2), ('SNORA62', 3), ('IFITM4P', 7), ('HLA-DRB6', 2), ('FUNDC2P2', 2), ('
19 input query terms found dup hits:	[('PPP1R2P1', 9), ('HSD17B7P2', 2), ('RPSAP9', 2), ('SNORD68', 2), ('SNORD38B', 2), ('RPL13AP3', 2),
15 input query terms found dup hits:	[('HNRNPA3P1', 2), ('LRP5L', 2), ('AGAP11', 2), ('CXADRP2', 3), ('CCDC144NL', 2), ('FAM66D', 3), ('F
18 input query terms found dup hits:	[('WASH2P', 2), ('MRS2P2', 2), ('SNORD29', 2), ('DPY19L2P4', 2), ('DPY19L2P1', 2), ('S100A7L2', 2), 
16 input query terms found dup hits:	[('PCNAP1', 2), ('SNORA63', 6), ('SERHL', 2), ('CEACAM22P', 2), ('SNORA16A', 2), ('FAM41AY1', 2), ('
18 input query terms found dup hits:	[('MBL1P', 2), ('SDHAP3', 2), ('PSORS1C3', 8), ('MYADML', 2), ('POM121L10P', 2), ('HLA-J', 9), ('HLA
16 input query terms found dup hit

In [48]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Mesothelioma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3H-AB3K-01,1,64,1,0.160908,-3.157426,-0.531035,0.483128,-0.152778,0.65969,0.001606,...,-0.068986,-0.086682,-0.061978,-0.900217,0.803567,-1.28139,-0.551677,-2.035927,0.175405,0.164067
TCGA-3H-AB3L-01,1,60,1,0.200408,-4.117426,-0.531035,0.786128,-0.033878,0.10959,-0.456594,...,-1.573486,-0.086682,-0.748878,-1.074117,0.139267,-0.39069,0.113323,0.166873,0.070905,0.942767
TCGA-3H-AB3M-01,1,53,0,-1.009592,-2.663326,-0.068835,0.828928,-0.737678,0.77649,-0.002594,...,1.636814,-0.086682,0.062822,-1.124917,0.242467,-0.46969,0.307823,-1.703127,-0.213695,1.932667
TCGA-3H-AB3O-01,1,58,1,-0.147492,-2.266526,-0.531035,0.274528,-0.311878,0.31179,-1.012494,...,-0.528386,-0.086682,1.031722,0.384783,3.061667,-1.28139,-0.580077,0.947073,1.408105,2.816367
TCGA-3H-AB3S-01,1,69,1,0.282008,-2.042126,-0.531035,0.825228,-0.210878,0.07039,-0.070994,...,-2.090786,-0.086682,-0.748878,-1.587117,-0.441533,-1.28139,0.417023,-0.154627,-0.004295,-0.111033


In [49]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 87 samples.


In [50]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Mesothelioma', the least common label is '1' with 87 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 64.0
  75%: 69.0
Min: 28
Max: 81
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 16 occurrences. This represents 18.39% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [37]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [39]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Mesothelioma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE68950',
 'GSE73162',
 'GSE51024',
 'GSE42977',
 'GSE64738',
 'GSE163720',
 'GSE67487',
 'GSE163722',
 'GSE99362',
 'GSE57464',
 'GSE172159',
 'GSE107754',
 'GSE117668',
 'GSE112154',
 'GSE73161',
 'GSE163721',
 'GSE40345',
 'GSE52788',
 'GSE131027']

Repeat the below steps for all the accession numbers

In [4]:
cohort = accession_num = "GSE131027"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Mesothelioma/GSE131027/GSE131027_family.soft.gz',
 '/home/yxcheng/GEO/Mesothelioma/GSE131027/GSE131027_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [5]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment target based on pathogeni

In [6]:
clinical_data.head()

,!Sample_geo_accession,GSM3759992,GSM3759993,GSM3759994,GSM3759995,GSM3759996,GSM3759997,GSM3759998,GSM3759999,GSM3760000,...,GSM3760074,GSM3760075,GSM3760076,GSM3760077,GSM3760078,GSM3760079,GSM3760080,GSM3760081,GSM3760082,GSM3760083
0,!Sample_characteristics_ch1,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,...,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy,tissue: tumor biopsy
1,!Sample_characteristics_ch1,cancer: Breast cancer,cancer: Colorectal cancer,cancer: Colorectal cancer,cancer: Bile duct cancer,cancer: Mesothelioma,cancer: Urothelial cancer,cancer: Pancreatic cancer,cancer: Melanoma,cancer: Colorectal cancer,...,cancer: Prostate cancer,cancer: Ovarian cancer,cancer: Mesothelioma,cancer: vulvovaginal,cancer: Colorectal cancer,cancer: Bile duct cancer,cancer: Prostate cancer,cancer: Colorectal cancer,cancer: Colorectal cancer,cancer: Colorectal cancer
2,!Sample_characteristics_ch1,mutated gene: ATR,mutated gene: FAN1,mutated gene: ERCC3,mutated gene: FANCD2,mutated gene: BAP1,mutated gene: DDB2,mutated gene: TP53,mutated gene: ATM,mutated gene: CHEK1,...,mutated gene: FAN1,mutated gene: MRE11,mutated gene: XPC,mutated gene: PMS2,mutated gene: ATM,mutated gene: PALB2,mutated gene: NBN,mutated gene: CEP57,mutated gene: CDH1,mutated gene: FANCM
3,!Sample_characteristics_ch1,predicted: HRDEXP: HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: HRD,...,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: NO_HRD,predicted: HRDEXP: HRD,predicted: HRDEXP: HRD
4,!Sample_characteristics_ch1,parp predicted: kmeans-2: PARP sensitive,parp predicted: kmeans-2: PARP sensitive,parp predicted: kmeans-2: PARP sensitive,parp predicted: kmeans-2: PARP sensitive,parp predicted: kmeans-2: PARP sensitive,parp predicted: kmeans-2: PARP sensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,...,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive,parp predicted: kmeans-2: PARP insensitive


In [7]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: tumor biopsy'],
 1: ['cancer: Breast cancer',
  'cancer: Colorectal cancer',
  'cancer: Bile duct cancer',
  'cancer: Mesothelioma',
  'cancer: Urothelial cancer',
  'cancer: Pancreatic cancer',
  'cancer: Melanoma',
  'cancer: Hepatocellular carcinoma',
  'cancer: Ovarian cancer',
  'cancer: Cervical cancer',
  'cancer: Head and Neck cancer',
  'cancer: Sarcoma',
  'cancer: Prostate cancer',
  'cancer: Adenoid cystic carcinoma',
  'cancer: NSCLC',
  'cancer: Oesophageal cancer',
  'cancer: Thymoma',
  'cancer: Others',
  'cancer: CUP',
  'cancer: Renal cell carcinoma',
  'cancer: Gastric cancer',
  'cancer: Neuroendocrine cancer',
  'cancer: vulvovaginal'],
 2: ['mutated gene: ATR',
  'mutated gene: FAN1',
  'mutated gene: ERCC3',
  'mutated gene: FANCD2',
  'mutated gene: BAP1',
  'mutated gene: DDB2',
  'mutated gene: TP53',
  'mutated gene: ATM',
  'mutated gene: CHEK1',
  'mutated gene: BRCA1',
  'mutated gene: WRN',
  'mutated gene: CHEK2',
  'mutated gene: BRCA2',


Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [8]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Mesothelioma\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Mesothelioma\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [10]:
is_gene_availabe = True
trait_row_id = 0
age_row_id = 2
gender_row_id = 1

trait_type = 'binary'

In [11]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [12]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue: Hippocampus':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, 23, 1)

In [13]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM6216198,GSM6216199,GSM6216200,GSM6216201,GSM6216202,GSM6216203,GSM6216204,GSM6216205,GSM6216206,GSM6216207,GSM6216208,GSM6216209,GSM6216210,GSM6216211,GSM6216212
Epilepsy,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
Age,23,29,37,26,16,13,62,58,63,68,77,59,50,39,23
Gender,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [14]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM6216198,GSM6216199,GSM6216200,GSM6216201,GSM6216202,GSM6216203,GSM6216204,GSM6216205,GSM6216206,GSM6216207,GSM6216208,GSM6216209,GSM6216210,GSM6216211,GSM6216212
0,cg00000029,0.717,0.643,0.645,0.699,0.587,0.676,0.566,0.608,0.592,0.739,0.550,0.469,0.674,0.605,0.576
1,cg00000108,0.974,0.978,0.969,0.974,0.969,0.964,0.968,0.973,0.955,0.973,0.953,0.953,0.950,0.971,0.977
2,cg00000109,0.710,0.825,0.802,0.864,0.791,0.770,0.870,0.876,0.817,0.880,0.843,0.829,0.845,0.856,0.854
3,cg00000165,0.294,0.210,0.246,0.214,0.193,0.186,0.240,0.212,0.337,0.228,0.309,0.214,0.270,0.203,0.273
4,cg00000236,0.923,0.903,0.945,0.873,0.957,0.936,0.877,0.906,0.893,0.910,0.868,0.901,0.924,0.911,0.872


In [15]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['cg00000029',
 'cg00000108',
 'cg00000109',
 'cg00000165',
 'cg00000236',
 'cg00000289',
 'cg00000292',
 'cg00000321',
 'cg00000363',
 'cg00000622',
 'cg00000658',
 'cg00000714',
 'cg00000721',
 'cg00000734',
 'cg00000769',
 'cg00000807',
 'cg00000884',
 'cg00000905',
 'cg00000924',
 'cg00000948']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [16]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']\n"

If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [18]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, '31717405', nan, nan, nan], 'AlleleB_ProbeSeq': [nan, 'ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAACCCCG', nan, nan, nan], 'Infinium_Design_Type': ['II', 'I', 'II', 'II', 'II'], 'Next_Base': [nan, 'A', nan, nan, nan], 'Color_Channel': [nan, 'Red', nan, nan, nan], 'Forward_Sequence': ['AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCTGCTCAGCTGGAAAC[CG]AATTTGAGATAAATTCAAGGGTCTATGTGGACAAGACTGCTAGTGTCTCTCTCTGGATTG', 'TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTG

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [19]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, '31717405', nan, nan, nan], 'AlleleB_P

In [20]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [21]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [22]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [23]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 15 samples.


In [24]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Epilepsy', the least common label is '1.0' with 6 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.5
  50% (Median): 39.0
  75%: 60.5
Min: 13.0
Max: 77.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Haoyang/Epilepsy/cohort_info.json


In [26]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [32]:
from utils import *

In [33]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [34]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [35]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [45]:
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
1,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
2,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
3,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
4,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [46]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [47]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [48]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [49]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [50]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 59.27% ± 1.43%


In [52]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 90.09% ± 2.36%


In [53]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [55]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 10 genes with non-zero coefficients associated with the trait 'Lung-Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
   Variable  Coefficient  Absolute Coefficient
    SNORA81     0.605617              0.605617
    SNORD89    -0.561992              0.561992
     OR4F17     0.468452              0.468452
SNORD115-26    -0.289358              0.289358
     OR2L1P    -0.278115              0.278115
    TBC1D3C     0.248994              0.248994
     DMRTA2    -0.194621              0.194621
     IFIT1B    -0.188779              0.188779
    PPIAL4E     0.150659              0.150659
     TTTY16    -0.082366              0.082366


/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [56]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 14 significant genes associated with the trait 'Lung-Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
   MEX3A     0.061062 2.147189e-07           0.003723
 SLC19A1    -0.038907 6.271788e-07           0.005437
  CRABP2     0.036440 1.633672e-06           0.009442
   HMGB3     0.042606 8.393633e-06           0.020791
 DEFB118     0.028296 7.138943e-06           0.020791
     IL3    -0.016966 5.077872e-06           0.020791
  FAM83A     0.043838 8.074414e-06           0.020791
    RCC1     0.041270 1.786386e-05           0.028158
    NUF2     0.068410 1.754965e-05           0.028158
    SPP1     0.032014 1.782137e-05           0.028158
    ETV4     0.037361 1.688133e-05           0.028158
   EPB42    -0.020447 2.157070e-05           0.031168
   CENPM    -0.051850 3.061145e-05           0.040829
 PLEKHN1     0.044331 3.372218

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
